In [49]:
import quimb.tensor as qtn
import numpy as np
from local_discriminant import disc_parts
import pickle
import os


Tasks

Easy backand switching (cpu, gpu) or (torch, cuda, numpy)

COMMENT EVERYTHING

In [50]:
def sh_weight(x,a=39):
    if x==0:
        x=10e-9
    return 1.j*(1-np.cos(a*x))/np.sinh(2*np.pi*x)


def ch_weight(x):
    return 1./np.cosh(2*np.pi*x)

In [52]:
L=4 #use even numbers
J=1
h=1.5
sv_tol=1e-8
delta_t=.08
final_t=3.2

a=33 #0.05 33 #0.1 #32
num_steps=int(final_t/delta_t)


I=np.eye(2)
X=np.array([[0,1],[1,0]])
Z=np.array([[1,0],[0,-1]])
ZZ=np.kron(Z,Z)
XI=np.kron(X,I)
IX=np.kron(I,X)

local_hamiltonians=[-J*ZZ+(h*IX+h*XI)/2]*(L-1)
local_hamiltonians[0]=-J*ZZ+h*XI+h/2*IX
local_hamiltonians[-1]=-J*ZZ+h/2*XI+h*IX

h2=-J*(np.kron(np.kron(Z,I),np.kron(Z,I))-np.kron(np.kron(I,Z),np.kron(I,Z)))
h1=h*(np.kron(X,I)-np.kron(I,X))

#ham_tp=qtn.LocalHam1D(L,H2=h2,H1=h1,cyclic=False)
#ham_tm=qtn.LocalHam1D(L,H2=-h2,H1=-h1,cyclic=False)

weight_func=lambda x: sh_weight(x,a)
tol=sv_tol
options={}
options['split_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel','max_bond':100}
options['tebd_opts']={'tol':tol,'order':4,'progbar':False}
options['comp_opts']={'method':'svd','cutoff':tol,'cutoff_mode':'rel'}


In [ ]:
from transformations import s_transform
from mps_generators import kraus_mps
import torch
def change_backend(x):
    return torch.tensor(x,dtype=torch.complex128)

kraus=kraus_mps(X,int(L/2),L)
kraus.apply_to_arrays()
s_transform()

In [53]:

def generate_parts(l):
    sca_xs=[]
    scd_xs=[]
    sca_zs=[]
    scd_zs=[]
    folder=f'DATA/new_data02_L{l}/'
    ham_tp=qtn.LocalHam1D(l,H2=h2,H1=h1,cyclic=False)
    ham_tm=qtn.LocalHam1D(l,H2=-h2,H1=-h1,cyclic=False)
    for i in range(int(l/2)):
        print('Site: ',i)
        #generate parts
        scd_x,sca_x=disc_parts(X,i,l,delta_t,num_steps,ham_tp,ham_tm,opts=options,gpu=False)
        sca_xs.append(sca_x)
        scd_xs.append(scd_x)
        #save
        if not os.path.exists(folder):
            os.makedirs(folder)
        with open(folder+'sca_xs.pkl','wb') as f:
            pickle.dump(sca_xs,f)
        with open(folder+'scd_xs.pkl','wb') as f:
            pickle.dump(scd_xs,f)
        #print_shape
        print('x_parts: ', [x.shape for x in scd_x.arrays])

        #generate parts
        scd_z,sca_z=disc_parts(Z,i,l,delta_t,num_steps,ham_tp,ham_tm,opts=options,gpu=False)
        sca_zs.append(sca_z)
        scd_zs.append(scd_z)
        #save
        with open(folder+'sca_zs.pkl','wb') as f:
            pickle.dump(sca_zs,f)
        with open(folder+'scd_zs.pkl','wb') as f:
            pickle.dump(scd_zs,f)                        
        #print shape
        print('z_parts: ', [z.shape for z in scd_z.arrays])

    
    

In [54]:
for l in [4,6,8,10,12,14,16,18,20]:
    print(l)
    generate_parts(l)

4
Site:  0
100%|■■■■■■■■■■■■■■■■■■■■|sca [[0 1]
 [1 0]]0 done
100%|■■■■■■■■■■■■■■■■■■■■|sa [[0 1]
 [1 0]]0 done
100%|■■■■■■■■■■■■■■■■■■■■|scsaa [[0 1]
 [1 0]]0 done
100%|■■■■■■■■■■■■■■■■■■■■|scasa [[0 1]
 [1 0]]0 done
x_parts:  [(4, 2, 2), (4, 14, 2, 2), (14, 4, 2, 2), (4, 2, 2)]
100%|■■■■■■■■■■■■■■■■■■■■|sca [[ 1  0]
 [ 0 -1]]0 done
100%|■■■■■■■■■■■■■■■■■■■■|sa [[ 1  0]
 [ 0 -1]]0 done
100%|■■■■■■■■■■■■■■■■■■■■|scsaa [[ 1  0]
 [ 0 -1]]0 done
100%|■■■■■■■■■■■■■■■■■■■■|scasa [[ 1  0]
 [ 0 -1]]0 done
z_parts:  [(4, 2, 2), (4, 6, 2, 2), (6, 4, 2, 2), (4, 2, 2)]
Site:  1
100%|■■■■■■■■■■■■■■■■■■■■|sca [[0 1]
 [1 0]]1 done
100%|■■■■■■■■■■■■■■■■■■■■|sa [[0 1]
 [1 0]]1 done
100%|■■■■■■■■■■■■■■■■■■■■|scsaa [[0 1]
 [1 0]]1 done
100%|■■■■■■■■■■■■■■■■■■■■|scasa [[0 1]
 [1 0]]1 done
x_parts:  [(4, 2, 2), (4, 16, 2, 2), (16, 4, 2, 2), (4, 2, 2)]
100%|■■■■■■■■■■■■■■■■■■■■|sca [[ 1  0]
 [ 0 -1]]1 done
100%|■■■■■■■■■■■■■■■■■■■■|sa [[ 1  0]
 [ 0 -1]]1 done
100%|■■■■■■■■■■■■■■■■■■■■|scsaa [[ 1  0]
 [ 0 -

KeyboardInterrupt: 